In [9]:
import cv2 #pip install opencv-python
import pandas as pd #pip install pandas
import os #no need to install
from pyzbar.pyzbar import decode #pip install pyzbar 
os.chdir("D:\LEARN\Python\Final_Project")

### Ex1: Create a student dataset with ID

In [36]:
student_data = pd.read_excel("DATA.xlsx", sheet_name = "Student data")
student_data.head()

,Student_ID,STUDENT,Student_Major,Student_Class
0,20040004,Nguyễn Quang Hào,DS,20DS
1,20040003,Nguyễn Văn A,DS,20DS
2,20040002,Phan Văn B,IBM,20IBM1
3,20040000,Lê Thị Tấm,CSE,20CSE
4,20040001,Trần Chín,THM,20THM1


### Ex2: Create a book dataset with code 

In [37]:
book_data = pd.read_excel("DATA.xlsx", sheet_name = "Book data")
book_data.head()

,CODE,BOOK,AUTHOR
0,NaN,Những Cuộc Phiêu Lưu Trong Kinh Doanh,John Brooks
1,NaN,Tư Duy Nhanh Và Chậm,Daniel Kahneman
2,NaN,Vào Trong Hoang Dã,Jon Krakauer
3,NaN,Tương Lai Sau Đại Dịch Covid,Jason Schenker
4,NaN,Yêu những điều không hoàn hảo,Hea Min


### Ex3: Scan infomation on Student Card

In [33]:
img = cv2.imread('STUDENT_CARD.jpg')
code = decode(img)
# result = list()
for i in code:
    id = i.data.decode('utf-8')

for i in range(len(student_data)):
    if int(id) == int(student_data.loc[i,'Student_ID']):
        print(student_data.loc[i])
# data = student_data.loc[0,'Student number']
# print(result)

Student_ID               20040004
STUDENT          Nguyễn Quang Hào
Student_Major                  DS
Student_Class                20DS
Name: 0, dtype: object


In [40]:
book = cv2.imread("BOOK_1.jpg")
book_code = decode(book)
for i in book_code:
    print(i)

Decoded(data=b'8936037799377', type='EAN13', rect=Rect(left=1160, top=1717, width=165, height=60), polygon=[Point(x=1160, y=1717), Point(x=1161, y=1757), Point(x=1162, y=1777), Point(x=1325, y=1776), Point(x=1325, y=1748), Point(x=1324, y=1722), Point(x=1323, y=1718)])
Decoded(data=b'9786049317279', type='EAN13', rect=Rect(left=1158, top=1660, width=166, height=40), polygon=[Point(x=1158, y=1661), Point(x=1158, y=1669), Point(x=1159, y=1699), Point(x=1324, y=1700), Point(x=1323, y=1676), Point(x=1322, y=1662), Point(x=1243, y=1660)])
